In [1]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

installations:
1. pip install synapseclient

In [2]:
import os
# CHANGE
gs_path_for_the_encode_pseudobulk_pipeline_table = "ENCODE_PSEUDOBULK_TABLE_INPUT"
local_folder = "/home/jupyter/eila"
os.makedirs(local_folder, exist_ok=True)
local_metadata_tsv = "proc.tsv"
# NO NEED TO CHANGE:
bucket_id = "fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901"
# if human - no need to change. taken from the worspace dashboard
gs_chrom_size_human = "gs://broad-buenrostro-pipeline-genome-annotations/IGVF_human_v43/GRCh38_EBV.chrom.sizes.tsv"


In [3]:
def remove_file(filename):
    if os.path.exists(filename):
        os.remove(filename)


In [4]:

%run synapse_utils.ipynb

Welcome, eila!



In [5]:
%run tsv_files_utils.ipynb

In [6]:
# read the proc.tsv object
# Create a subset file of each rna_dataset and save it under the rna_dataset and call it {rna_dataset}_metadata.tsv

# TODO: copy the metadata file for ENCODE datasets
local_metadata_file_location = os.path.join(local_folder,local_metadata_tsv)
if os.path.exists(local_metadata_file_location) == False:
    print("execute cmd: gsutil cp gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/ENCODE_synpase_annotation_mapping/proc.tsv {}".format(local_metadata_file_location))
    !gsutil cp gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/ENCODE_synpase_annotation_mapping/proc.tsv $local_metadata_file_location
else:
    print("file proc.tsv is available, nothing to copy")

file proc.tsv is available, nothing to copy


In [7]:
import csv
from itertools import islice
import pandas as pd
metadata_col_names = None
df_relevant_rows = None


with open(local_metadata_file_location, "r", newline="") as tsvfile:
    reader = csv.reader(tsvfile, delimiter="\t")  # Set delimiter as tab

# Column names: ['cell_type_name,"rna_dataset","rna_library","rna_barcode","atac_dataset","atac_library","atac_barcode","file"']    # Read the header row
    metadata_col_names = next(reader)
    print("Column names:", metadata_col_names," type is {}".format(type(metadata_col_names)))
    metadata_col_names = metadata_col_names[0].split(",")
    print("metadata_col_names after split is {}".format(metadata_col_names))
    metadata_col_names = [x.strip('"') for x in metadata_col_names]
    print("metadata_col_names after strip is {}".format(metadata_col_names))
    print("Column names after processing is:", metadata_col_names," type is {}".format(type(metadata_col_names)))
    column_atac_dataset_index = metadata_col_names.index("atac_dataset")
    print("column_atac_dataset_index is {}".format(column_atac_dataset_index))

    rows = []  # To store the extracted rows
    for row in reader: # to do: un-comment
        # Process each row
        # print("row is {}. type is {}. len is {}".format(row,type(row),len(row)))
        row = row[0].split(",")
        row = [x.strip('"') for x in row]
        column_atac_dataset_value = row[column_atac_dataset_index]
        
        if column_atac_dataset_value != 'NA':
            rows.append(row)
            # print("adding row {}".format(row))
        else:
            # print("Nothing to add: column_atac_dataset_value is {}".format(column_atac_dataset_value))
            continue
    # Create a dataframe from the extracted rows
    df_encode_rows = pd.DataFrame(rows, columns=metadata_col_names)


Column names: ['cell_type_name,"rna_dataset","rna_library","rna_barcode","atac_dataset","atac_library","atac_barcode","file"']  type is <class 'list'>
metadata_col_names after split is ['cell_type_name', '"rna_dataset"', '"rna_library"', '"rna_barcode"', '"atac_dataset"', '"atac_library"', '"atac_barcode"', '"file"']
metadata_col_names after strip is ['cell_type_name', 'rna_dataset', 'rna_library', 'rna_barcode', 'atac_dataset', 'atac_library', 'atac_barcode', 'file']
Column names after processing is: ['cell_type_name', 'rna_dataset', 'rna_library', 'rna_barcode', 'atac_dataset', 'atac_library', 'atac_barcode', 'file']  type is <class 'list'>
column_atac_dataset_index is 4


In [8]:
df_encode_rows.head(2)

,cell_type_name,rna_dataset,rna_library,rna_barcode,atac_dataset,atac_library,atac_barcode,file
0,Hepatocytes,NA,NA,NA,ENCSR114OKU,ENCLB431YMH,GGAGGATCAAGCGGTA,EAID_000002
1,Hepatocytes,NA,NA,NA,ENCSR114OKU,ENCLB431YMH,GTTACGATCGTGGTAT,EAID_000002


In [9]:
len(df_encode_rows['atac_dataset'].to_list())

1236715

In [10]:
# gsutil_ls_txt.txt was generated with gsutils ls gs://path/to/fragmetn/files.tsv.gz
!ls gsutil_ls_txt*

gsutil_ls_txt.txt


In [11]:
# LOAD the ls file from local - does not work on this machine
def load_gs_ls_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines]
    return lines

# Example usage
file_path = 'gsutil_ls_txt.txt'
gs_path_to_search_sample_id = load_gs_ls_text_file(file_path)
print(gs_path_to_search_sample_id[0:5])


['gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/8ef75ac6-1a49-4cbe-b61d-d62af4c904d7/share/24e3d191-38aa-4ed1-9255-dbfb14161097/call-atac/wf_atac/3d2abad2-1ff0-4477-a7a8-35aef31e2a3e/call-align/ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI

In [12]:
gs_atac_dataset = [x.split("/")[-1].split(".")[0] for x in gs_path_to_search_sample_id]

In [13]:
atac_dataset_list = df_encode_rows["atac_dataset"].tolist()
atac_dataset_list[0:3]

['ENCSR114OKU', 'ENCSR114OKU', 'ENCSR114OKU']

In [14]:
# atac dataset in both
atac_dataset_to_fragment = list(set(gs_atac_dataset) & set(atac_dataset_list))
atac_dataset_to_fragment[0:6]

['ENCSR868ZEI',
 'ENCSR397LYX',
 'ENCSR453TVZ',
 'ENCSR845QFX',
 'ENCSR455CVZ',
 'ENCSR367GKP']

In [15]:
len(atac_dataset_to_fragment)

23

In [16]:
# this will check for which fragments do we have annotation data
df_relevant_rows = df_encode_rows[df_encode_rows['atac_dataset'].isin(atac_dataset_to_fragment)]
df_relevant_rows.head(2)

,cell_type_name,rna_dataset,rna_library,rna_barcode,atac_dataset,atac_library,atac_barcode,file
510368,macrophage,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTGACATAT,ENCSR987PQH,ENCLB714RNW,GTCACCATGAGCTATT,EAID_000093
510369,cardiomyocyte,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTTACATCC,ENCSR987PQH,ENCLB714RNW,ATGTTGTTGAGCTATT,EAID_000093


In [17]:
gs_path_to_fragment = list(filter(lambda x: any(substring in x for substring in atac_dataset_to_fragment), gs_path_to_search_sample_id))
print(gs_path_to_fragment[0:4])


['gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/ff3a5809-0cd2-464c-8a29-9bf6311111ae/share/02c2df11-8d6c-4786-868a-94ecf3858bc5/call-atac/wf_atac/26b48087-2f17-469d-b3ff-b64720f9ddb4/call-align/ENCSR367GKP

In [18]:
len(gs_path_to_fragment)

23

In [19]:
gs_path_to_fragment[0]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz'

In [20]:
# # os.path.basename("gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz")
# df_relevant_rows[df_relevant_rows['atac_dataset']=='ENCSR987PQH']['file'].to_list()[0]
# cell_type_id_names

In [ ]:
# %run tsv_files_utils.ipynb

# List of column names
pipeline_column_names = ['dataset_id_cell_id', 'cell_type_id', 'gs_path','chromosome']
# Initialize dataframe with column names
df_pseudobulk_pipeline= pd.DataFrame(columns=pipeline_column_names)


# Tanjin, here you decide how many samples to run
# where you decide to run on samples [0:1] or more
for fragment_file in gs_path_to_fragment:
    print("fragment_file is {}".format(fragment_file))
    # Step 1: split the labels file by cell_type_id. create tmp tsv files
    # Step 2: Split the fragment file by the cell_bracodes that are avaiable
    # Step 3: call the TagAlign for each of the fragmetns file
    # Step 4: save the TSV file 
    # Step 5: write the created file to bucket
    # Step 6: add a rows to the pipeline TSV table
    cur_atac_dataset = fragment_file.split("/")[-1].split(".")[0]
    print("cur_atac_dataset is {}".format(cur_atac_dataset))
    
    # Step 1: input: labels file. output: df of the subset by cell_type_id
    # Step 1-1: get the labels file from synpase.
    dataset_id = 'syn34271785'
    folder_name = df_relevant_rows[df_relevant_rows['atac_dataset']==cur_atac_dataset]['file'].to_list()[0]
    print("folder_name in synpase is {}".format(folder_name))
    # (starting_folder_id, first_subfolder_name, second_subfolder_name):
    file_id_list = get_file_names_in_second_subfolder(dataset_id,folder_name,"labels")
    file_id = file_id_list[0] # the label's file synID
    print('file_id is {}'.format(file_id))
    
    # Download the synpase label file:
    # Obtain a pointer and download the data to full_labels_file_path
    labels_file_path = os.path.join(local_folder,"{}_labels".format(file_id))
    label_file_synHandle = syn.get(entity=file_id,downloadLocation=labels_file_path) 
    print('label_file_synHandle is {}'.format(label_file_synHandle))
    full_labels_file_path = label_file_synHandle.path
    print('full_labels_file_path is {}'.format(full_labels_file_path))
    
    # Load the labels data to dataframe
    df_labels = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_labels_file_path)
    print("df_labels head is {}".format(df_labels.head()))
    # df,fld_name,col_name,prefix_file_name)
    split_files_dir = os.path.join(local_folder,cur_atac_dataset)
    local_files_created = split_df_based_on_col_name(df_labels,split_files_dir,"cell_type_id","{}_labels".format(cur_atac_dataset))
    print("\n\nlocal_files_created based on cell_type_id {}".format(local_files_created))

    # Step 2: split the fragments files based on the files under the local_files_created
    # loop on all the cell_type_id and create a fragment file that will be copied under the cell_type_id_folder
    cell_type_id_names = os.listdir(split_files_dir)
    print("number of cell_type_id is {}. cell_type_id_names is {}".format(len(cell_type_id_names),cell_type_id_names))

    for cell_type_id in cell_type_id_names[0:1]:
        print("split_files_dir is {}".format(split_files_dir))
        print("cell_type_id is {}".format(cell_type_id))
        full_label_cell_type_file_name = local_files_created[cell_type_id]
        print("full_label_cell_type_file_name is {}".format(full_label_cell_type_file_name))
        # load the file for the cell_type_id into a dataframe and extract the cell_barcode 
        # the file name is from ls the folder
        cell_type_labels_df = read_tsv_file(full_label_cell_type_file_name)
        print("cell_type_df.head(2) is {}".format(cell_type_labels_df.head()))
        # get the cell barcode
        cells_barcodes = cell_type_labels_df['cell_id'].to_list()
        print("cells_barcodes[0:5] is {}".format(cells_barcodes[0:5]))

        # Load the fragment file
        print("!!!gs fragment_file is {}".format(fragment_file))
        local_file_name = os.path.basename(fragment_file)
        full_local_file_name = os.path.join(local_folder,local_file_name)
        print("full_local_file_name is {}".format(full_local_file_name))
        if (os.path.exists(full_local_file_name) == False):
            !gsutil cp $fragment_file $full_local_file_name
        else:
            print("Nothing to copy. all files available")
        
        # Define column names
        column_names = ['chr', 'start_pos', 'end_pos', 'cell_id_atac_dataset' ,'read_count']
        # file_path,column_names
        # Tanjin, for any change / debug purpose: you can edit read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names
        # to load only 100 lines. This will shorten the execution time
        # For any change in the utils file, make sure to execute %run tsv_files_utils.ipynb or %run synapse_utils.ipynb
        print("read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names for file is {} and column_names {}".format(full_local_file_name,column_names))
        df_fragments_file = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names(full_local_file_name,column_names)
        print("df_fragments_file is {}".format(df_fragments_file.head(5)))

        # create all the options of cell_id and dataset_id that might be used to identify cell_id
        # it is not always aligned with the fragment file cell_id
        print("split_column_to_cell_id_and_atac_dataset")
        df_fragments_cell_id_and_atac_col_addition = split_column_to_cell_id_and_atac_dataset(df_fragments_file)
        print("df_fragments_cell_id_and_atac_col_addition is {}".format(df_fragments_cell_id_and_atac_col_addition.head(100)))
        print("DataFrame df_fragments_cell_id_and_atac_col_addition size: {} ", df_fragments_cell_id_and_atac_col_addition.shape)

        # Tanjin: potential issue.
        # cell_id in the labels file can be a comnination of the atac_dataset_cell_id: ENCSR100UGC_ACCATTAAGCCGCAAC
        # or cell_id_atac_dataset ACCATTAAGCCGCAAC_ENCSR100UGC or something else.
        # all the combinations are ready in the df_fragments_cell_id_and_atac_col_addition
        df_fragments_cell_type_id = filter_dataframe(df_fragments_cell_id_and_atac_col_addition,'atac_dataset_cell_id',cells_barcodes)
        print("df_fragments_cell_type_id is {}".format(df_fragments_cell_type_id))
        print("DataFrame df_fragments_cell_type_id size: ", df_fragments_cell_type_id.shape)

        if (df_fragments_cell_type_id.empty):
            print("!!! ERR!!! df_fragments_cell_type_id is empty for cell type {}".format(cell_type_id))
            
        # step 3: call tagAlign
        df_tag_align = tag_align(df_fragments_cell_type_id)
        print("df_tag_align.head() is {}".format(df_tag_align.head()))
        # Step 4: save the tagAlign file
        tagAlign_file_name = "{}_{}.tsv".format(cur_atac_dataset,cell_type_id)
        print("tagAlign_file_name is {}".format(tagAlign_file_name))
        full_path_cell_type_fragment_file = os.path.join(os.path.dirname(full_label_cell_type_file_name),tagAlign_file_name)
        print("full_path_cell_type_fragment_file is {}".format(full_path_cell_type_fragment_file))
        df_tag_align.to_csv(full_path_cell_type_fragment_file, sep='\t', index=False)

        # Step 5: copy file to bucket - under the original fragment file
        gs_dest = os.path.join(os.path.dirname(fragment_file),tagAlign_file_name)
        print("copy tagAlign file {} to gcp  {}".format(full_path_cell_type_fragment_file,gs_dest))
        # CHECK IF WORKING WITH FILE NAME ONLY
        !gsutil -m cp $full_path_cell_type_fragment_file $gs_dest
        print("finish copy tagAlign file {} to gcp  {}".format(full_path_cell_type_fragment_file,gs_dest))
        
        # Step 6: add a rows to the pipeline TSV table
        # all fragment file cell_ids, cell_type_id, path_to_gs_file, chromosome
        # chromosome_size_human - taken from the worskapce dashboard
        df_pseudobulk_pipeline = pd.concat([df_pseudobulk_pipeline, \
                                            pd.DataFrame({'dataset_id_cell_id': [tagAlign_file_name.strip(".tsv")], \
                                                          'cell_type_id': [cell_type_id], \
                                                          'gs_path':[gs_dest], \
                                                          'chromosome': [gs_chrom_size_human]})], \
                                                          ignore_index=True)
        
        print("!!!!df_pseudobulk_pipeline is {} ".format(df_pseudobulk_pipeline))

# save the pipeline input table
local_pseudobulk_pipeline_table = os.path.join(local_folder,"{}_pipeline_table.tsv".format(cur_atac_dataset))
print("local_pseudobulk_pipeline_table is {}".format(local_pseudobulk_pipeline_table))
df_pseudobulk_pipeline.to_csv(local_pseudobulk_pipeline_table, sep='\t', index=False)
print("finsihed copy {}".format(local_pseudobulk_pipeline_table))
gs_pseudobulk_pipeline_table = os.path.join("gs://",bucket_id,gs_path_for_the_encode_pseudobulk_pipeline_table)
print("gs_pseudobulk_pipeline_table is  {}".format(gs_pseudobulk_pipeline_table))
!gsutil cp $local_pseudobulk_pipeline_table $gs_pseudobulk_pipeline_table
print("finished copying to {}".format(gs_pseudobulk_pipeline_table))


\ [1 files][794.2 MiB/794.2 MiB]   36.0 MiB/s                                   
Operation completed over 1 objects/794.2 MiB.                                    
read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names for file is /home/jupyter/eila/ENCSR453TVZ.atac.filter.fragments.hg38.tsv.gz and column_names ['chr', 'start_pos', 'end_pos', 'cell_id_atac_dataset', 'read_count']
df_fragments_file is     chr  start_pos  end_pos                cell_id_atac_dataset  read_count
0  chr1      10007    10279  CCGTACTTGCTACGTGCAGTAT_ENCSR453TVZ           1
1  chr1      10007    10413  TTGGACCCAGGTTACCCTCGGA_ENCSR453TVZ           1
2  chr1      10024    10394  TGGTAGGTCAGGTAGTCGAAAC_ENCSR453TVZ           1
3  chr1      10031    10126  AGTCTCCTTACTTAGCAAAGAG_ENCSR453TVZ           1
4  chr1      10037    10126  AGTCTCCTTACTTAGCAAAGAG_ENCSR453TVZ           1
split_column_to_cell_id_and_atac_dataset
df_fragments_cell_id_and_atac_col_addition is      chr  start_pos  end_pos       

In [ ]:
local_files_created

In [ ]:
# !ls -lt /home/jupyter/eila/ENCSR987PQH/fibroblast

In [ ]:
# !rm -r /home/jupyter/eila/ENCSR987PQH/

In [ ]:
# this is from the label file
# !cat /home/jupyter/eila/ENCSR987PQH/fibroblast/split_fragment_file_fibroblast.tsv | more  
    

In [ ]:
!ls

In [ ]:
# pwd

In [ ]:
# !rm split_fragment_file_ENCSR987PQH.tsv